<a href="https://colab.research.google.com/github/hwangsaeyeon/dacon/blob/main/lgbm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

train_df = pd.read_csv("/content/drive/MyDrive/mood/train.csv",index_col=0)
test_df = pd.read_csv("/content/drive/MyDrive/mood/test.csv",index_col=0)

In [3]:
train = pd.DataFrame(train_df)
train

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,engnat,age,hand,religion,orientation,voted,married,familysize,ASD,nerdiness
index,,,,,,,,,,,,,,,,,,,,,
0,1.0,5.0,5.0,5.0,1.0,4.0,5.0,5.0,1.0,3.0,...,1.0,20,2.0,12.0,4.0,2.0,1.0,4.0,2.0,1
1,4.0,4.0,4.0,4.0,4.0,5.0,4.0,4.0,3.0,3.0,...,1.0,49,1.0,2.0,1.0,1.0,2.0,4.0,2.0,1
2,4.0,5.0,5.0,4.0,3.0,5.0,5.0,5.0,4.0,4.0,...,2.0,43,1.0,2.0,2.0,2.0,3.0,4.0,2.0,1
3,4.0,4.0,4.0,2.0,4.0,3.0,3.0,5.0,3.0,4.0,...,1.0,17,2.0,1.0,1.0,2.0,1.0,2.0,2.0,1
4,4.0,4.0,4.0,4.0,3.0,3.0,4.0,2.0,3.0,4.0,...,2.0,18,2.0,12.0,1.0,2.0,1.0,1.0,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,2.0,5.0,4.0,3.0,3.0,4.0,4.0,4.0,3.0,4.0,...,1.0,17,1.0,1.0,3.0,2.0,1.0,3.0,2.0,0
14996,5.0,4.0,5.0,4.0,4.0,5.0,5.0,4.0,4.0,5.0,...,2.0,45,1.0,3.0,1.0,1.0,2.0,3.0,2.0,1
14997,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,5.0,...,1.0,20,1.0,1.0,2.0,1.0,1.0,3.0,1.0,1


## **Null값 전처리**

null값 확인

In [43]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15000 entries, 0 to 14999
Data columns (total 69 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Q1            14959 non-null  float64
 1   Q2            14931 non-null  float64
 2   Q3            14950 non-null  float64
 3   Q4            14929 non-null  float64
 4   Q5            14962 non-null  float64
 5   Q6            14952 non-null  float64
 6   Q7            14924 non-null  float64
 7   Q8            14952 non-null  float64
 8   Q9            14944 non-null  float64
 9   Q10           14928 non-null  float64
 10  Q11           14941 non-null  float64
 11  Q12           14933 non-null  float64
 12  Q13           14960 non-null  float64
 13  Q14           14964 non-null  float64
 14  Q15           14955 non-null  float64
 15  Q16           14967 non-null  float64
 16  Q17           14963 non-null  float64
 17  Q18           14937 non-null  float64
 18  Q19           14947 non-nu

q1~q26 null값 전처리

In [44]:
#train에서 question에 대한 값이 아예 없는 row삭제(8개)
cols = train.columns
train = train.dropna(how='all',subset =cols[0:26])
train

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,engnat,age,hand,religion,orientation,voted,married,familysize,ASD,nerdiness
index,,,,,,,,,,,,,,,,,,,,,
0,1.0,5.0,5.0,5.0,1.0,4.0,5.0,5.0,1.0,3.0,...,1.0,20,2.0,12.0,4.0,2.0,1.0,4.0,2.0,1
1,4.0,4.0,4.0,4.0,4.0,5.0,4.0,4.0,3.0,3.0,...,1.0,49,1.0,2.0,1.0,1.0,2.0,4.0,2.0,1
2,4.0,5.0,5.0,4.0,3.0,5.0,5.0,5.0,4.0,4.0,...,2.0,43,1.0,2.0,2.0,2.0,3.0,4.0,2.0,1
3,4.0,4.0,4.0,2.0,4.0,3.0,3.0,5.0,3.0,4.0,...,1.0,17,2.0,1.0,1.0,2.0,1.0,2.0,2.0,1
4,4.0,4.0,4.0,4.0,3.0,3.0,4.0,2.0,3.0,4.0,...,2.0,18,2.0,12.0,1.0,2.0,1.0,1.0,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,2.0,5.0,4.0,3.0,3.0,4.0,4.0,4.0,3.0,4.0,...,1.0,17,1.0,1.0,3.0,2.0,1.0,3.0,2.0,0
14996,5.0,4.0,5.0,4.0,4.0,5.0,5.0,4.0,4.0,5.0,...,2.0,45,1.0,3.0,1.0,1.0,2.0,3.0,2.0,1
14997,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,5.0,...,1.0,20,1.0,1.0,2.0,1.0,1.0,3.0,1.0,1


최빈값 구하기

In [45]:
#데이터 인덱스를 nerdiness가 1인 것과 0인것으로 나눔
idx_one = []
idx_zero = []
for i in train.index:
  if train['nerdiness'][i]==1:
    idx_one.append(i)
  elif train['nerdiness'][i]==0:
    idx_zero.append(i)

from pandas._libs.algos import pad
from scipy.stats import mode

cols = list(train.columns)

def mode_val(feature,idx):
  return mode(train[feature][idx])

mode_val_one = []
mode_val_zero = []

#idx_one
for i in cols:
  mode_val_one.append(mode_val(i,idx_one)[0][0])

#idx_zero
for i in cols:
  mode_val_zero.append(mode_val(i,idx_zero)[0][0])

In [51]:
#question null값을 최빈값으로 바꾸기 
col_q = train.columns[0:26]
q_mode_val_zero = mode_val_zero[0:26]
q_mode_val_one = mode_val_one[0:26]
idx=0
for j in col_q:
  for i in train.index:
    if train[j].isna().loc[i] == True:
      if i in idx_one:
        train[j].loc[i] = q_mode_val_one[idx]
      if i in idx_zero:
        train[j].loc[i] = q_mode_val_zero[idx]
  idx+=1

KeyboardInterrupt: ignored

TIPI 값 전처리

In [ ]:
#TIPI값을 최빈값으로 바꾸기 
col_t = train.columns[30:40]
t_mode_val_zero = mode_val_zero[30:40]
t_mode_val_one = mode_val_one[30:40]
idx=0
for j in col_t:
  for i in train.index:
    if train[j].isna().loc[i] == True:
      if i in idx_one:
        train[j].loc[i] = t_mode_val_one[idx]
      if i in idx_zero:
        train[j].loc[i] = t_mode_val_zero[idx]
  idx+=1

In [ ]:
train.info()

engnat null값 전처리 

In [ ]:
engnat1 = ['GBR','CAN','USA','IRL','AUS','SWE','NLD'] #영어가 모국어인 국가
engnat2 = ['DEU','POL','PHL','MYS'] #영어가 모국어가 아닌 국가 

In [ ]:
for i in train.index:
  if train['engnat'].isna()[i] == True:
    if train['country'][i] in engnat1:
      train['engnat'][i] = 1
    elif train['country'][i] in engnat2:
      train['engnat'][i] = 2
    else:
      train['engnat'][i] = 2

In [ ]:
train.info()

Hand, Religion, Orientation, Familysize, Married, ASD null값 전처리

In [ ]:
for i in train.index:
  if train['hand'].isna()[i] == True:
    train['hand'] = 1

for i in train.index:
  if train['religion'].isna()[i] == True:
    train['religion'] = 2

for i in train.index: #400개 널값
  if train['orientation'].isna()[i] == True:
    train['orientation'] = 1

for i in train.index: #400개널값
  if train['familysize'].isna()[i] == True:
    train['familysize'] = 2

for i in train.index:
  if train['married'].isna()[i] == True:
    train['married'] = 1

for i in train.index:
  if train['ASD'].isna()[i] == True:
    train['ASD'] = 2

for i in train.index:
  if train['voted'].isna()[i] == True:
    train['voted'] = 2

for i in train.index:
  if train['gender'].isna()[i] == True:
    train['gender'] = 2

education - 0

In [ ]:
for i in train.index:
  if train['education'].isna()[i] == True:
    train['education'] = 0

In [ ]:
final_train = train.drop('country',axis=1)

## 모델

In [14]:
!pip install eli5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 216 kB 3.7 MB/s 
     |████████████████████████████████| 133 kB 50.0 MB/s 
  Created wheel for eli5: filename=eli5-0.13.0-py2.py3-none-any.whl size=107748 sha256=3051313589a95ce3b477d67c379708792ff94ae2670b8c9e3d57eaac30c69c84
  Stored in directory: /root/.cache/pip/wheels/cc/3c/96/3ead31a8e6c20fc0f1a707fde2e05d49a80b1b4b30096573be
Successfully built eli5
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires Jinja2<3.0,>=2.10.1, but you have jinja2 3.1.2 which is incompatible.


In [4]:
train = pd.DataFrame(train_df)
train

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,engnat,age,hand,religion,orientation,voted,married,familysize,ASD,nerdiness
index,,,,,,,,,,,,,,,,,,,,,
0,1.0,5.0,5.0,5.0,1.0,4.0,5.0,5.0,1.0,3.0,...,1.0,20,2.0,12.0,4.0,2.0,1.0,4.0,2.0,1
1,4.0,4.0,4.0,4.0,4.0,5.0,4.0,4.0,3.0,3.0,...,1.0,49,1.0,2.0,1.0,1.0,2.0,4.0,2.0,1
2,4.0,5.0,5.0,4.0,3.0,5.0,5.0,5.0,4.0,4.0,...,2.0,43,1.0,2.0,2.0,2.0,3.0,4.0,2.0,1
3,4.0,4.0,4.0,2.0,4.0,3.0,3.0,5.0,3.0,4.0,...,1.0,17,2.0,1.0,1.0,2.0,1.0,2.0,2.0,1
4,4.0,4.0,4.0,4.0,3.0,3.0,4.0,2.0,3.0,4.0,...,2.0,18,2.0,12.0,1.0,2.0,1.0,1.0,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,2.0,5.0,4.0,3.0,3.0,4.0,4.0,4.0,3.0,4.0,...,1.0,17,1.0,1.0,3.0,2.0,1.0,3.0,2.0,0
14996,5.0,4.0,5.0,4.0,4.0,5.0,5.0,4.0,4.0,5.0,...,2.0,45,1.0,3.0,1.0,1.0,2.0,3.0,2.0,1
14997,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,5.0,...,1.0,20,1.0,1.0,2.0,1.0,1.0,3.0,1.0,1


In [5]:
#train에서 question에 대한 값이 아예 없는 row삭제(8개)
cols = train.columns
train = train.dropna(how='all',subset =cols[0:26])
train

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,engnat,age,hand,religion,orientation,voted,married,familysize,ASD,nerdiness
index,,,,,,,,,,,,,,,,,,,,,
0,1.0,5.0,5.0,5.0,1.0,4.0,5.0,5.0,1.0,3.0,...,1.0,20,2.0,12.0,4.0,2.0,1.0,4.0,2.0,1
1,4.0,4.0,4.0,4.0,4.0,5.0,4.0,4.0,3.0,3.0,...,1.0,49,1.0,2.0,1.0,1.0,2.0,4.0,2.0,1
2,4.0,5.0,5.0,4.0,3.0,5.0,5.0,5.0,4.0,4.0,...,2.0,43,1.0,2.0,2.0,2.0,3.0,4.0,2.0,1
3,4.0,4.0,4.0,2.0,4.0,3.0,3.0,5.0,3.0,4.0,...,1.0,17,2.0,1.0,1.0,2.0,1.0,2.0,2.0,1
4,4.0,4.0,4.0,4.0,3.0,3.0,4.0,2.0,3.0,4.0,...,2.0,18,2.0,12.0,1.0,2.0,1.0,1.0,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,2.0,5.0,4.0,3.0,3.0,4.0,4.0,4.0,3.0,4.0,...,1.0,17,1.0,1.0,3.0,2.0,1.0,3.0,2.0,0
14996,5.0,4.0,5.0,4.0,4.0,5.0,5.0,4.0,4.0,5.0,...,2.0,45,1.0,3.0,1.0,1.0,2.0,3.0,2.0,1
14997,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,5.0,...,1.0,20,1.0,1.0,2.0,1.0,1.0,3.0,1.0,1


In [6]:
train = train.drop(['country'],axis=1)

In [7]:
#train_question = train.columns[0:26]
#for i in train_question:   
#  train[i].fillna(value=train[i].mean(), inplace=True)

In [8]:
train = train.drop(['introelapse','testelapse','surveyelapse','age','religion','age'],axis=1)

In [9]:
train  = train.fillna(0)

In [10]:
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=0).fit_resample(train.iloc[:,0:-1], train['nerdiness'])

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_resampled,y_resampled , test_size = 0.2)

In [521]:
train.columns

Index(['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Q11',
       'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Q18', 'Q19', 'Q20', 'Q21',
       'Q22', 'Q23', 'Q24', 'Q25', 'Q26', 'TIPI1', 'TIPI2', 'TIPI3', 'TIPI4',
       'TIPI5', 'TIPI6', 'TIPI7', 'TIPI8', 'TIPI9', 'TIPI10', 'VCL1', 'VCL2',
       'VCL3', 'VCL4', 'VCL5', 'VCL6', 'VCL7', 'VCL8', 'VCL9', 'VCL10',
       'VCL11', 'VCL12', 'VCL13', 'VCL14', 'VCL15', 'VCL16', 'education',
       'urban', 'gender', 'engnat', 'hand', 'orientation', 'voted', 'married',
       'familysize', 'ASD', 'nerdiness'],
      dtype='object')

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics    

forest = RandomForestClassifier(n_estimators=1000)
forest.fit(X_train, y_train)
 
y_pred = forest.predict(X_test)

print('정확도 :', metrics.accuracy_score(y_test, y_pred))

정확도 : 0.8129518072289157


In [15]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(forest, random_state=1).fit(X_test, y_test)
eli5.show_weights(perm, feature_names = X_test.columns.tolist())

Weight,Feature
0.0151 ± 0.0049,Q6
0.0116 ± 0.0055,Q15
0.0083 ± 0.0055,Q2
0.0075 ± 0.0042,Q1
0.0058 ± 0.0023,Q14
0.0044 ± 0.0016,Q18
0.0041 ± 0.0015,Q12
0.0036 ± 0.0035,Q11
0.0031 ± 0.0014,Q23
0.0029 ± 0.0025,TIPI7


In [497]:
print(forest.score(X_train,y_train))

1.0


In [16]:
import numpy as np
np.random.seed(3)
from lightgbm import LGBMClassifier
import warnings

warnings.filterwarnings('ignore')
lgbm_clf = LGBMClassifier(
            n_estimators=1000,
            learning_rate = 0.1,
            objective = 'binary',
        )
lgbm_clf.fit(X_train, y_train)

LGBMClassifier(n_estimators=1000, objective='binary')

In [17]:
y_pred = lgbm_clf.predict(X_test)
#Confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
#Accuracy
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred,y_test)
print(cm)
print(accuracy)

[[1309  369]
 [ 265 1377]]
0.8090361445783133


In [18]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(lgbm_clf, random_state=1).fit(X_test, y_test)
eli5.show_weights(perm, feature_names = X_test.columns.tolist())

Weight,Feature
0.0216 ± 0.0049,Q6
0.0192 ± 0.0046,Q15
0.0173 ± 0.0041,Q1
0.0133 ± 0.0064,TIPI7
0.0094 ± 0.0037,Q18
0.0077 ± 0.0057,Q11
0.0077 ± 0.0049,Q14
0.0073 ± 0.0038,Q2
0.0067 ± 0.0037,Q5
0.0062 ± 0.0026,TIPI10


In [512]:
from xgboost import XGBClassifier
from sklearn import metrics    
model = XGBClassifier(n_estimators=1000, learning_rate=0.1, max_depth=4, random_state = 32)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
print('정확도 :', metrics.accuracy_score(y_test, y_pred))

정확도 : 0.7942771084337349
